# Training the UNET

First, we import the libraries we need for the training.

In [1]:
import numpy as np
import tensorflow as tf
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from glob import glob

from tensorflow.keras import initializers, regularizers, constraints, optimizers, metrics
from tensorflow.keras import backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.layers import Dense, Input, Conv2D, Conv2DTranspose, Dropout, Flatten, BatchNormalization, Concatenate, Lambda, Activation, Reshape, Layer, InputSpec
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence

import Data_Generator.DataGenerator as DG
import Loss_Metrics.LossMetrics 
import UNet_Arch.UNET_architecture as unet

tf.keras.backend.clear_session()

We set the path and the names of the data set.

In [2]:
dim = 128  # Available dimensions: 128, 256, 512

# Allow for overriding in tests
dim = os.environ.get("UNET_SHEZEN_DIM", dim)

data_url = {128: '0e1e5d/dl/Shezen_128128.zip', 256: '3a4a44/dl/Shezen_256256.zip', 512: '80a4eb/dl/Shezen_512512.zip'}

if not os.path.exists(f"./Shezen_{dim:d}{dim:d}"):
    !  wget https://pandora.infn.it/public/{data_url[dim]} -qO tmp.zip
    ! unzip -q tmp.zip 
    ! rm -rf tmp.zip

path='./Shezen_{:d}{:d}/'.format(dim,dim) # input images path
data_path= path + '*'
data_list = [os.path.basename(f) for f in glob(data_path)] 

train_list = data_list[0:500]
val_list = data_list[500:]

training_gen = DG.DataGenerator(path=path,list_X=train_list,batch_size=8,dim=(dim,dim))
validation_gen = DG.DataGenerator(path=path,list_X=val_list,batch_size=1,dim=(dim,dim))

In [3]:
checkpoint_path="../results/weights-{epoch:03d}-{loss:.2f}-{DSC:.2f}.hdf5" # intermediate weight save path
checkpoint_path_val="../results/weights-val-{epoch:03d}-{loss:.2f}-{val_DSC:.2f}.hdf5"  # intermediate weigth save path

check=ModelCheckpoint(filepath=checkpoint_path, monitor='loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', save_freq='epoch') # Checkpoints parameters
check_val=ModelCheckpoint(filepath=checkpoint_path_val, monitor='val_DSC', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
adamlr = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, amsgrad=True)
reduce_lr = ReduceLROnPlateau(monitor='val_DSC', factor=0.5, patience=8, min_lr=0.000005, verbose= True)


met = tf.keras.metrics.MeanIoU(num_classes=2)

In [4]:
unet_test=unet.U_net((dim,dim,1))
unet_test.compile(loss=Loss_Metrics.LossMetrics.DSC_loss,optimizer=adamlr, metrics=[Loss_Metrics.LossMetrics.DSC])
unet_test.summary()
history = unet_test.fit(training_gen, validation_data= validation_gen, epochs=50,callbacks=[reduce_lr, check_val, check],verbose=1)

with open('history.json', 'w') as f:
    json.dump(str(history.history), f)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 8)  16          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 8)  32         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              